In [1]:
def is_colab():
    """Check if the code is running in Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Use the function to conditionally run magic commands
if is_colab():
    # Run Colab-specific magic commands
    print("Running in Colab, executing magic commands.")
    !rm -rf microproyecto3NLP/
    !git clone https://github.com/cjohana031/microproyecto3NLP
    !cp -R microproyecto3NLP/* .
    # Add any other Colab-specific setup
else:
    # Alternative setup for non-Colab environments
    print("Not running in Colab, nothing else is needed.")

Running in Colab, executing magic commands.
Cloning into 'microproyecto3NLP'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 34.20 MiB | 31.01 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
with open('data/test.txt', 'r') as f:
  print(f.read())

holii


In [8]:
!pip install datasets sentencepiece

     -------------------------------------- 991.5/991.5 kB 4.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from huggingface_hub import login
token = input("Enter your Hugging Face token: ")
login(token=token) # Token de Hugging Face

In [10]:
# Importación de librerías
import sentencepiece as spm
import io
import time
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from datasets import load_dataset
#from peft import LoraConfig, get_peft_model, TaskType
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,BertForQuestionAnswering
)

# Configurar semillas para facilitar la reproducibilidad de los resultados
seed = 99
torch.manual_seed(seed)
np.random.seed(seed)

In [9]:
#read dataset test

test_path = 'data/test-00000-of-00001.parquet'
test = pd.read_parquet(test_path)
test = test[test['article'].str.len() < 800]
test



,example_id,article,answer,question,options
45,high18377.txt,A new law helps people with disabilities. The ...,D,According to the passage we can see that_.,[it will be difficult for the normal persons t...
46,high18377.txt,A new law helps people with disabilities. The ...,C,Ramps can help people_.,"[with hearing problems, who have difficulty in..."
47,high18377.txt,A new law helps people with disabilities. The ...,D,"The word ""Braille"" means_.","[words known by foreigners, words known by com..."
48,high18377.txt,A new law helps people with disabilities. The ...,C,What do you think the disabled would feel abou...,"[They may think the new law reasonable., Some ..."
49,high18377.txt,A new law helps people with disabilities. The ...,B,The fact that the new law has been passed show...,[the government has paid no attention to the d...
...,...,...,...,...,...
4915,middle926.txt,"Most People don't like mice, but they love one...",C,"In the cartoon, Pluto is a _ dog.","[clever, smart, foolish, bad]"
4916,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,A,Who is the oldest of the four boys?,"[Dick., David., Dean., Denis.]"
4917,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,D,What do the four boys like doing every day?,"[Reading stories., Playing computer games., Wa..."
4918,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,C,What will they do on Mrs. Black's birthday?,"[Eat out., Play soccer., Have a party., Play c..."


In [ ]:
#read dataset train

train_path = 'data/train-00000-of-00001.parquet'
train = pd.read_parquet(train_path)
train = train[train['article'].str.len() < 800]
train['options']

244      [in the evening, in the afternoon, in the morn...
245      [They were busy working in the control room., ...
246      [The air hostess was humorous  ., The old lady...
329      [5,000 dollars goes to a computer!, New invent...
330      [That he didn't win the$5,000., That he hadn't...
                               ...                        
87829       [a map, a cook book, a story book, a magazine]
87846                        [in, next to, near, far from]
87847                  [by bike, by train, by air, by bus]
87848    [he doesn't like her, he can see her every day...
87849                           [Tom, Betty, Sophia, Jane]
Name: options, Length: 8930, dtype: object

In [ ]:
# Cargar modelo y tokenizador

model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Configurar tokenizador
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

c:\Users\cjoha\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cjoha\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [27]:
import json
from datasets import load_dataset
from transformers import (
    pipeline
)
from tqdm import tqdm

# Format RACE example for LLama inference
def format_race_prompt(row):
    """
    Format a RACE example from a pandas DataFrame row into a prompt suitable for LLama.
    
    Args:
        row: A row from the pandas DataFrame containing RACE data
    
    Returns:
        A formatted prompt string
    """
    article = row["article"]
    question = row["question"]
    options = row["options"]
    
    # Format options as A, B, C, D
    formatted_options = ""
    for i, opt in enumerate(options):
        option_letter = chr(65 + i)  # A, B, C, D
        formatted_options += f"{option_letter}. {opt}\n"
    
    prompt = f"""Read the following passage and answer the question by choosing the correct option.

Passage:
{article}

Question: {question}

Options:
{formatted_options}

The correct answer is:"""
    
    return prompt

# Convert LLama's output to a letter choice (A, B, C, D)
def extract_answer(output_text):
    """
    Extract the model's answer from the output text.
    
    Args:
        output_text: The model's raw output
    
    Returns:
        A single letter (A, B, C, or D) representing the model's answer
    """
    # First, look for a direct letter mention
    output_text = output_text.strip().upper()
    
    # Simple extraction approach - take the first letter that's A, B, C, or D
    for char in output_text:
        if char in "ABCD":
            return char
    
    # If no direct letter was found
    return None

# Evaluate the model on a subset of RACE
def evaluate_llama_on_race(model, tokenizer, race_df):
    """
    Evaluate a LLama model on a DataFrame containing RACE dataset examples.
    
    Args:
        model: The LLama model
        tokenizer: The model's tokenizer
        race_df: Pandas DataFrame containing RACE dataset examples
    
    Returns:
        Accuracy score and detailed results
    """
    # Set up text generation pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=10,  # We only need a short answer
        do_sample=False     # Deterministic generation
    )
    
    # Evaluate
    print("Evaluating model...")
    results = []
    correct = 0
    total = 0
    
    # Iterate through DataFrame rows
    for _, row in tqdm(race_df.iterrows(), total=len(race_df)):
        # Create prompt
        prompt = format_race_prompt(row)
        
        # Get model prediction
        output = pipe(prompt)[0]["generated_text"]
        predicted_answer = extract_answer(output.replace(prompt, ""))
        
        # Get correct answer
        correct_idx = row["answer"]
        correct_letter = chr(65 + ord(correct_idx) - ord('A'))
        
        # Check if prediction is correct
        is_correct = predicted_answer == correct_letter
        if is_correct:
            correct += 1
        total += 1
        
        # Store result
        results.append({
            "passage": row["article"],
            "question": row["question"],
            "options": row["options"].tolist(),
            "ground_truth": correct_letter,
            "prediction": predicted_answer,
            "is_correct": is_correct
        })
    
    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0
    print(f"Accuracy: {accuracy:.4f} ({correct}/{total})")
    
    return accuracy, results

# Save evaluation results
def save_results(results, accuracy, output_file="llama_race_results.json"):
    """
    Save evaluation results to a JSON file.
    
    Args:
        results: List of result dictionaries
        accuracy: Overall accuracy score
        output_file: Path to save the results
    """
    output_data = {
        "accuracy": accuracy,
        "results": results
    }
    
    with open(output_file, 'w') as f:
        json.dump(output_data, f, indent=2)
    
    print(f"Results saved to {output_file}")

# Example usage:
# 1. Load the model and tokenizer
# model_id = "meta-llama/Llama-2-7b-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

accuracy, results = evaluate_llama_on_race(model,tokenizer, train.head(100))
print(accuracy, results)


Device set to use cpu


Evaluating model...


100%|██████████| 100/100 [09:36<00:00,  5.77s/it]

Accuracy: 0.3400 (34/100)
0.34 [{'passage': 'The air hostess   was in a small kitchen at the back of the plane, preparing the plates for lunch, when a little old lady came and spoke to her, "Could you please tell me," she asked, "where is the ladies\' lavatory   in the plane?"\n"Yes, madam," said the air hostess and smiled. "It is right at the other end of the plane---at the front."\nThe little lady went too far. She walked all the way to the front of the plane, opened the door in front of her, and saw the captain of the plane and the other officers. They were all busy with their work and did not see her. She went out again, shut the door and returned to the air hostess.\n"Oh, didn\'t you find it, madam?" the girl asked her. "Yes, I did," said the little lady. "But there are four men in the ladies\' lavatory watching television."', 'question': 'The story happened  _  .', 'options': ['in the evening', 'in the afternoon', 'in the morning', 'at midnight'], 'ground_truth': 'C', 'prediction

In [35]:
# Format RACE example for LLama inference
def format_race_prompt(row):
    """
    Format a RACE example from a pandas DataFrame row into a prompt suitable for LLama.
    
    Args:
        row: A row from the pandas DataFrame containing RACE data
    
    Returns:
        A formatted prompt string
    """
    article = row["article"]
    question = row["question"]
    options = row["options"]
    
    # Format options as A, B, C, D
    formatted_options = ""
    for i, opt in enumerate(options):
        option_letter = chr(65 + i)  # A, B, C, D
        formatted_options += f"{option_letter}. {opt}\n"
    
    prompt = f"""Read the following passage and answer the question by choosing the correct option.

Passage:
{article}

Question: {question}

Options:
{formatted_options}

The correct answer is:"""
    
    return prompt

def evaluate_llama_on_race_with_logits(model, tokenizer, race_df, device="cuda"):
    """
    Evaluate a LLama model on RACE using logits and softmax directly.
    
    Args:
        model: The LLama model
        tokenizer: The model's tokenizer
        race_df: Pandas DataFrame containing RACE dataset examples
        device: Device to run inference on ("cuda" or "cpu")
    
    Returns:
        Accuracy score and detailed results
    """
    model.eval()  # Set model to evaluation mode
    
    results = []
    correct = 0
    total = 0
    
    # Get token IDs for the letters A, B, C, D
    letter_token_ids = {}
    for letter in "ABCD":
        letter_tokens = tokenizer.encode(f" {letter}", add_special_tokens=False)
        if len(letter_tokens) == 1:
            letter_token_ids[letter] = letter_tokens[0]
        else:
            # If encoding a single letter results in multiple tokens, use the first one
            # This can happen with some tokenizers
            letter_token_ids[letter] = letter_tokens[0]
    
    print(f"Letter token IDs: {letter_token_ids}")
    
    # Iterate through DataFrame rows
    for _, row in tqdm(race_df.iterrows(), total=len(race_df)):
        # Create prompt
        prompt = format_race_prompt(row)
        
        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        # Get model outputs without sampling (greedy decoding) - we just need the logits
        with torch.no_grad():
            outputs = model(**inputs, return_dict=True)
        
        # Get logits for the last token (where the model would predict A, B, C, or D)
        last_token_logits = outputs.logits[0, -1, :]
        
        # Extract logits for each answer option (A, B, C, D)
        answer_logits = {letter: last_token_logits[token_id].item() 
                          for letter, token_id in letter_token_ids.items()}
        
        # Apply softmax to get probabilities
        logits_tensor = torch.tensor([answer_logits[letter] for letter in "ABCD"])
        probabilities = F.softmax(logits_tensor, dim=0).tolist()
        
        # Get the predicted answer (letter with highest probability)
        predicted_letter = "ABCD"[torch.argmax(logits_tensor).item()]
        
        # Get correct answer
        correct_idx = row["answer"]
        correct_letter = chr(65 + ord(correct_idx) - ord('A'))
        
        # Check if prediction is correct
        is_correct = predicted_letter == correct_letter
        if is_correct:
            correct += 1
        total += 1
        
        # Store detailed result
        results.append({
            "passage": row["article"],
            "question": row["question"],
            "options": row["options"].tolist() if hasattr(row["options"], "tolist") else row["options"],
            "ground_truth": correct_letter,
            "prediction": predicted_letter,
            "probabilities": {letter: float(prob) for letter, prob in zip("ABCD", probabilities)},
            "logits": {letter: float(answer_logits[letter]) for letter in "ABCD"},
            "is_correct": is_correct
        })
    
    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0
    print(f"Accuracy: {accuracy:.4f} ({correct}/{total})")
    
    return accuracy, results

# Alternative implementation with a slightly different approach to extracting token logits
def evaluate_llama_on_race_with_next_token_logits(model, tokenizer, race_df, device="cuda"):
    """
    Evaluate LLama by getting logits for next token prediction of A, B, C, or D.
    
    This is a more robust method that tokenizes the prompt + each possible answer letter
    and compares the likelihoods directly.
    
    Args:
        model: The LLama model
        tokenizer: The model's tokenizer
        race_df: Pandas DataFrame containing RACE dataset examples
        device: Device to run inference on ("cuda" or "cpu")
    
    Returns:
        Accuracy score and detailed results
    """
    model.eval()  # Set model to evaluation mode
    
    results = []
    correct = 0
    total = 0
    
    # Iterate through DataFrame rows
    for _, row in tqdm(race_df.iterrows(), total=len(race_df)):
        # Create prompt
        prompt = format_race_prompt(row)
        
        # Calculate likelihood for each possible answer
        answer_logits = {}
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
        with torch.no_grad():
            # Get model outputs for the prompt
            outputs = model(input_ids)
            # Get logits for the last token position
            next_token_logits = outputs.logits[0, -1, :]
        for letter in "ABCD":
            # Create input with the letter as the completion
            completion_ids = tokenizer(f" {letter}", add_special_tokens=False, return_tensors="pt").input_ids.to(device)
            
            # Get likelihood of the completion
            with torch.no_grad():
                completion_logit = next_token_logits[completion_ids[0, 0]].item()
                answer_logits[letter] = completion_logit
        
        # Convert to tensor and apply softmax
        logits_tensor = torch.tensor([answer_logits[letter] for letter in "ABCD"])
        probabilities = F.softmax(logits_tensor, dim=0).tolist()
        
        # Get predicted answer (letter with highest probability)
        predicted_letter = "ABCD"[torch.argmax(logits_tensor).item()]
        
        # Get correct answer
        correct_idx = row["answer"]
        correct_letter = chr(65 + ord(correct_idx) - ord('A'))
        
        # Check if prediction is correct
        is_correct = predicted_letter == correct_letter
        if is_correct:
            correct += 1
        total += 1
        
        # Store result
        results.append({
            "passage": row["article"],
            "question": row["question"],
            "options": row["options"].tolist() if hasattr(row["options"], "tolist") else row["options"],
            "ground_truth": correct_letter,
            "prediction": predicted_letter,
            "probabilities": {letter: float(prob) for letter, prob in zip("ABCD", probabilities)},
            "logits": {letter: float(answer_logits[letter]) for letter in "ABCD"},
            "is_correct": is_correct
        })
    
    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0
    print(f"Accuracy: {accuracy:.4f} ({correct}/{total})")
    
    return accuracy, results

# Save evaluation results
def save_results(results, accuracy, output_file="llama_race_logits_results.json"):
    """
    Save evaluation results to a JSON file.
    
    Args:
        results: List of result dictionaries
        accuracy: Overall accuracy score
        output_file: Path to save the results
    """
    output_data = {
        "accuracy": accuracy,
        "results": results
    }
    
    with open(output_file, 'w') as f:
        json.dump(output_data, f, indent=2)
    
    print(f"Results saved to {output_file}")

accuracy, results = evaluate_llama_on_race_with_next_token_logits(model, tokenizer, train.head(100), "cpu")
print(accuracy, results)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [07:11<00:00,  4.31s/it]

Accuracy: 0.3400 (34/100)
0.34 [{'passage': 'The air hostess   was in a small kitchen at the back of the plane, preparing the plates for lunch, when a little old lady came and spoke to her, "Could you please tell me," she asked, "where is the ladies\' lavatory   in the plane?"\n"Yes, madam," said the air hostess and smiled. "It is right at the other end of the plane---at the front."\nThe little lady went too far. She walked all the way to the front of the plane, opened the door in front of her, and saw the captain of the plane and the other officers. They were all busy with their work and did not see her. She went out again, shut the door and returned to the air hostess.\n"Oh, didn\'t you find it, madam?" the girl asked her. "Yes, I did," said the little lady. "But there are four men in the ladies\' lavatory watching television."', 'question': 'The story happened  _  .', 'options': ['in the evening', 'in the afternoon', 'in the morning', 'at midnight'], 'ground_truth': 'C', 'prediction

In [30]:
save_results(results, accuracy)

Results saved to llama_race_logits_results.json
